In [91]:
df=spark.sql("select * from default.test_json_parse")

In [4]:
df.show()

+--------------------+
|               jdata|
+--------------------+
|{  "cars": {    "...|
|{  "cars": {    "...|
|{  "cars": {    "...|
|{  "cars": {    "...|
|{  "cars": {    "...|
+--------------------+



In [58]:
from pyspark.sql.functions import *

In [7]:
from pyspark.sql.types import *

In [97]:
schema_car=StructType([StructField("cars", StringType(), True)])
schema_cars=StructType([StructField("car1", StringType(), True),
                       StructField("car2", StringType(), True),
                        StructField("car3", StringType(), True)
                       
                       ])

In [88]:
#{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }

In [86]:
df1=df.select(from_json('jdata',schema_car).alias('cars')).\
select('cars.*').select(from_json('cars',schema_cars).alias('cars_info'))\
.select('cars_info.*')

In [87]:
df1.show()

+----+----+----+
|car1|car2|car3|
+----+----+----+
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
+----+----+----+



In [147]:
#'{"array_1": [{"a": "1", "b": "1"}],"array_2": [{"a": "2", "b": "2"}],"array_3": [{"a": "3", "b": "3"}]}'

In [146]:
df_a=spark.sql("select * from default.test_json_array_parse")
#df_a.show(truncate=False)
schema_array=StructType([StructField("array_1", StringType(), True),
                       StructField("array_2", StringType(), True),
                        StructField("array_3", StringType(), True)
                       
                       ])

schema_array_data=StructType([StructField("a", StringType(), True),
                       StructField("b", StringType(), True)
                       ])



In [145]:
df1_a=df_a.select(from_json('jadata',schema_array).alias('aaaa')).select('aaaa.*').\
    select(from_json('array_1',schema_array_data).alias('array_1'),\
    from_json('array_2',schema_array_data).alias('array_2'),\
    from_json('array_3',schema_array_data).alias('array_3')).\
select('array_1.*','array_2.*','array_3.*')
df1_a.show()

+---+---+---+---+---+---+
|  a|  b|  a|  b|  a|  b|
+---+---+---+---+---+---+
|  1|  1|  2|  2|  3|  3|
|  1|  1|  2|  2|  3|  3|
|  1|  1|  2|  2|  3|  3|
|  1|  1|  2|  2|  3|  3|
|  1|  1|  2|  2|  3|  3|
+---+---+---+---+---+---+

